<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/Optimizaci%C3%B3nHiperpar%C3%A1metros/OptunaLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación Optuna

In [2]:
!pip install Optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.4 MB/s eta 0:00:00


# Búsqueda de hiperparámetros Optuna

In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import time
import psutil
from sklearn.metrics import mean_squared_error
start_time = time.time()

url1 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data1 = pd.read_csv(url1)
data1.drop(['Unnamed: 0'], axis=1, inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data1)
X = scaled_data[:, data1.columns != 'Inputs']
y = scaled_data[:, data1.columns == 'Inputs']

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

def objective(trial):
    time_steps = trial.suggest_int('time_steps', 1, 10)
    X_lstm, y_lstm = create_dataset(X, y, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

    model = Sequential()
    model.add(LSTM(units=trial.suggest_categorical('units', [50, 100, 150]),
                   return_sequences=trial.suggest_categorical('return_sequences', [True, False]),
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    if trial.suggest_categorical('return_sequences', [True, False]):
        model.add(LSTM(units=trial.suggest_categorical('units_2', [50, 100, 150])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=trial.suggest_categorical('optimizer', ['adam', 'rmsprop']))

    model.fit(X_train, y_train, epochs=trial.suggest_int('epochs', 20, 35, 50), batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]), verbose=3)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print('Mejores hiperparámetros:', study.best_trial.params)
print('Mejor resultado:', study.best_value)


process = psutil.Process()
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
execution_time = end_time - start_time

minutes = int(execution_time // 60)
seconds = int(execution_time % 60)

print(f"Tiempo de ejecución: {minutes:02d}:{seconds:02d}")


Epoch 2/20
